In [6]:
 
import numpy as np 

  

# P1 = [120.7, 69.7, 280] # RAI

# P2 = [138.3, 274.2, 70.6] # LAI

P1 = [-120.7, -69.7, -280] # LPS 

P2 = [138.3, -274.2, -70.6] # LPS 

Origin = np.minimum(P1,P2) 

ROIExtent = np.absolute(np.subtract(P1,P2)) 

DesiredSpacing = 0.38 

DesiredSize  = ROIExtent / DesiredSpacing #= round 

DesiredSizeR = np.around(DesiredSize) 

  

print('Size:', DesiredSizeR) 

print('Origin:', Origin) 

print('Spacing:', DesiredSpacing) 

print(ROIExtent) 

print(DesiredSize) 

 

Size: [682. 538. 551.]
Origin: [-120.7 -274.2 -280. ]
Spacing: 0.38
[259.  204.5 209.4]
[681.57894737 538.15789474 551.05263158]


In [5]:
### This Code reads in a defined recentered ZUNU_vida-lung-recentered.nii.gz
### and finds the bounds of the lungs to Upsample the actual image given by the Nifti.gz file
import SimpleITK as sitk
import numpy as np
import nibabel as nib

# IMPORTANT: ASSIGN THE IMAGE, MASK, AND IMAGE OUTPUT PATHS BEFORE CONTINUING
# Note: Output mask path is optional to save recentered Mask
mask_path = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-6-MESAL-7013450-E3/Vida_Masks/ZUNU_vida-lung.img.gz"
image_path = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-6-MESAL-7013450-E3/MESAL-7013450-E3_FRC_80kV_0.75mm_Qr40d.nii.gz"
output_image_path = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-6-MESAL-7013450-E3/Markups/Vx3(6).nrrd"
output_mask_path = "z:/D-Images/MESAL-3/2-Checked-Done-Temp/Done/Case-6-MESAL-7013450-E3/Vida_Masks/ZUNU_vida-lung-recentered.img.gz"

# Here is the function to perform the Image Upsample
# Variables Needed:
# User Inputs: (3 Needed, 1 optional) Listed Below. 
# Input Image Path ((.nii.gz) file), Mask Path ((".img.gz")), Output Image path 
# Outputs: A New file named "Vx3.nrrd" in the Output Image Path

# This function is called near the end of the code
def resample_image(input_image, new_size, output_origin, output_spacing, output_pixel_type, interpolator):
        # Create a resampler object
        resampler = sitk.ResampleImageFilter()
        #Set the interpolator
        resampler.SetInterpolator(interpolator)
        # Set the new output size, origin, and spacing
        resampler.SetSize(new_size)
        resampler.SetOutputOrigin(output_origin)
        resampler.SetOutputSpacing(output_spacing)
        # Set the output pixel type
        resampler.SetOutputPixelType(output_pixel_type)
        # Perform the resampling
        output_image = resampler.Execute(input_image)
        return(output_image)

# This function will turn indices into physical point locations for analysis
# It will be called twice in the program 
def Physical_Point(x, y, z):
    index = [x, y, z]
    p1 = image.TransformIndexToPhysicalPoint((x, y, z))
    point1 = (p1[0], p1[1], p1[2])
    return(point1)

# Read image with Simple ITK
image = sitk.ReadImage(image_path)
mask = sitk.ReadImage(mask_path)
print("Image and Mask Loaded!")

#Flip and Center the Mask Image
vnda = sitk.GetArrayFromImage(mask)
vnds = np.flip(vnda, axis=0)
fvess = sitk.GetImageFromArray(vnds)
fvess.CopyInformation(image)
print("Mask Flipped and Recentered!")


# convert image array to numpy array
transposed_array = sitk.GetArrayFromImage(image)
transposed_mask = sitk.GetArrayFromImage(fvess)
print("Image and Mask Arrays Created!")

# Find indicies of mask where lungs are
indices_0 = np.argwhere(transposed_mask == 0)
indices_20 = np.argwhere(transposed_mask == 20)
indices_30 = np.argwhere(transposed_mask == 30)

#Find Max and Min points
max = indices_20.max(axis=0)
min = indices_20.min(axis=0)
max1 = indices_30.max(axis=0)
min1 = indices_30.min(axis=0)

print("Indices Found...")

print("Max20 is ", max)
print("Max30 is ", max1)
print("Min20 is ", min)
print("Min30 is ", min1)

# For Case 42 I find these maxes/minumums
# Max20 is  [521 431 499]
# Max30 is  [506 439 276]
# Min20 is  [ 55  71 249]
# Min30 is  [34 77 40]

# The points below this line (min and max) are calculated using lines 50-95
# min = [34, 71, 40] 
# max = [521, 439, 499] 
maxxc = max[2]
maxxc1 = max1[2]
if maxxc > maxxc1:
    maxx = maxxc
if maxxc < maxxc1:
    maxx = maxxc1
minxc = min[2]
minxc1 = min1[2]
if minxc < minxc1:
    minx = minxc
if minxc > minxc1:
    minx = minxc1
maxyc = max[1]
maxyc1 = max1[1]
if maxyc > maxyc1:
    maxy = maxyc
if maxyc < maxyc1:
    maxy = maxyc1
minyc = min[1]
minyc1 = min1[1]
if minyc < minyc1:
    miny = minyc
if minyc > minyc1:
    miny = minyc1
maxzc = max[0]
maxzc1 = max1[0]
if maxzc > maxzc1:
    maxz = maxzc
if maxzc < maxzc1:
    maxz = maxzc1
minzc = min[0]
minzc1 = min1[0]
if minzc < minzc1:
    minz = minzc
if minzc > minzc1:
    minz = minzc1
print("Maximum and Minimum Points Created")

# Values in sitk.resampleimagefilter need to be Integers not Floats
x = int(minx)
y = int(miny)
z = int(minz)

maximum = [x, y, z]
print("Maximum =", maximum)
# Find physical point of minimum
point1 = Physical_Point(x, y, z)

i = int(maxx)
j = int(maxy)
k = int(maxz)
minimum = [i, j, k]
print("Minimum =", minimum)
# Find physical point of max
point2 = Physical_Point(i, j, k)
print("Physical Points Created!")

# Print Physical Points to see them
print("Point 1:", point1)
print("Point 2:", point2)

# Code to make Original Vx3
output_origin = np.minimum(point1,point2) 
ROIExtent = np.absolute(np.subtract(point1,point2)) 
DesiredSpacing = 0.38 

x1 = ROIExtent[0] / DesiredSpacing  
y1 = ROIExtent[1] / DesiredSpacing
z1 = ROIExtent[2] / DesiredSpacing

# Turn Floats into Integers and add a boundary
x2 = int(x1) + 10
y2 = int(y1) + 10
z2 = int(z1) + 10

# Setting Variables before calling function        
output_pixel_type = sitk.sitkInt16
output_spacing = (.38, .38, .38)
interpolator = sitk.sitkBSpline3
output_size = (x2, y2, z2)

# Perform the Upsample
output_image = resample_image(image, output_size, output_origin, output_spacing, output_pixel_type, interpolator)
sitk.WriteImage(output_image, output_image_path)

# Save Recentered Mask 
# sitk.WriteImage(fvess, output_mask_path)

print("Vx3 was created!")
print("Output Image Dimensions: ", output_size)
print("Output Image Spacing: ", output_spacing)
print("Output Image Origin: ", output_origin)
print("Interpolator: BSpline")


Image and Mask Loaded!
Mask Flipped and Recentered!
Image and Mask Arrays Created!
Indices Found...
Max20 is  [516 395 460]
Max30 is  [502 399 282]
Min20 is  [ 77 158 279]
Min30 is  [ 94 159  77]
Maximum and Minimum Points Created
Maximum = [77, 158, 77]
Minimum = [460, 399, 516]
Physical Points Created!
Point 1: (-110.865234375, -213.556640625, -258.20001220703125)
Point 2: (127.013671875, -63.873046875, -38.70001220703125)
Vx3 was created!
Output Image Dimensions:  (635, 403, 587)
Output Image Spacing:  (0.38, 0.38, 0.38)
Output Image Origin:  [-110.86523438 -213.55664062 -258.20001221]
Interpolator: BSpline
